In [ ]:
import os
import cv2
from PIL import Image as Img
from numpy import asarray, expand_dims
from keras_facenet import FaceNet
import numpy as np
import pickle
from mtcnn.mtcnn import MTCNN
import tensorflow as tf

In [ ]:
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = FaceNet()

In [ ]:
liveness_model = tf.keras.models.load_model('liveness.model')

In [ ]:
folder = 'UserImages/'  # Create a folder to store user images
user_database = {}

In [ ]:
def is_live(image):
    # Resize the image to the input size expected by the liveness model
    input_size = (32, 32)
    image = cv2.resize(image, input_size)

    # Preprocess the image (e.g., normalize pixel values)
    image = image.astype("float32") / 255.0

    # Expand dimensions to match the input shape of the model
    image = np.expand_dims(image, axis=0)

    # Perform liveness detection prediction
    prediction = liveness_model.predict(image)

    # Determine if the face is live or not based on the prediction
    if prediction[0][0] > 0.5:  # You may need to adjust the threshold
        return True
    else:
        return False

def capture_and_train_user(username, num_images=20):
    user_images = []  # Store user's face images
    print(f"Capturing images for user: {username}")

    # Create a directory for the user if it doesn't exist
    user_dir = os.path.join(folder, username)
    os.makedirs(user_dir, exist_ok=True)

    # Initialize the webcam
    cap = cv2.VideoCapture(0)

    image_count = 0

    # Initialize the MTCNN face detection model
    detector = MTCNN()

    while image_count < num_images:
        _, frame = cap.read()
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect faces using MTCNN
        faces = detector.detect_faces(frame_rgb)

        for result in faces:
            x, y, w, h = result['box']

            # Crop and resize the detected face region
            face = frame_rgb[y:y + h, x:x + w]
            face = Img.fromarray(face)
            face = face.resize((160, 160))
            face_array = asarray(face)
            face_array = expand_dims(face_array, axis=0)

            # Perform liveness detection
            if is_live(face_array[0]):
                user_images.append(face_array)

                # Draw a rectangle around the detected face
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                image_count += 1

        cv2.imshow('Capture User Images', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Compute embeddings for user images
    user_embeddings = [MyFaceNet.embeddings(image) for image in user_images]

    # Store the user's embeddings in the database
    user_database[username] = user_embeddings

    # Save the user database to a file
    with open("user_database.pkl", "wb") as db_file:
        pickle.dump(user_database, db_file)

    cap.release()
    cv2.destroyAllWindows()
    print(f"{num_images} images for user {username} captured and trained.")

if __name__ == "__main__":
    username = input("Enter the username of the user: ")
    capture_and_train_user(username, num_images=20)  # Adjust the number of images as needed

In [ ]:
# Load the user database
with open("user_database.pkl", "rb") as db_file:
    user_database = pickle.load(db_file)

In [ ]:
def is_live(image):
    # Resize the image to the input size expected by the liveness model
    input_size = (32, 32)
    image = cv2.resize(image, input_size)

    # Preprocess the image (e.g., normalize pixel values)
    image = image.astype("float32") / 255.0

    # Expand dimensions to match the input shape of the model
    image = np.expand_dims(image, axis=0)

    # Perform liveness detection prediction
    prediction = liveness_model.predict(image)

    # Determine if the face is live or not based on the prediction
    if prediction[0][0] > 0.5:  # You may need to adjust the threshold
        return True
    else:
        return False

def recognize_user(face_embedding, face_image):
    min_dist = 100  # Initialize minimum distance
    recognized_user = None  # Initialize recognized user

    for username, embeddings in user_database.items():
        for user_embedding in embeddings:
            dist = np.linalg.norm(user_embedding - face_embedding)
            if dist < min_dist:
                min_dist = dist
                recognized_user = username

    if min_dist < 0.7:
        # Perform liveness detection on the recognized face
        if is_live(face_image):
            return recognized_user
        else:
            return "Live Person Detected (Not Authorized)"
    else:
        return "Unknown User"

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = HaarCascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame_rgb[y:y + h, x:x + w]
        face = Img.fromarray(face)
        face = face.resize((160, 160))
        face_array = asarray(face)
        face_array = expand_dims(face_array, axis=0)

        face_embedding = MyFaceNet.embeddings(face_array)

        recognized_user = recognize_user(face_embedding, face_array[0])

        # Display recognized user or "Unknown User"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_thickness = 2
        color = (0, 255, 0) if recognized_user != "Unknown User" else (0, 0, 255)
        cv2.putText(frame, recognized_user, (x, y - 10), font, font_scale, color, font_thickness, cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    cv2.imshow('Face Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()